In [39]:
import sys
sys.path.append('../src')

import os
import requests
import json
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta

from betfair import Betfair

pd.set_option('display.max_columns', None)

In [40]:
# need to import daily data so that we can get the race ids and loop through for results.

In [41]:
app_key = 'ABGJLOlKaLtTsMIp'

username1 = input("Email:")
password1 = input("Password:")

In [42]:
trading = betfairlightweight.APIClient(
        username1, password1, app_key
    )
trading.login_interactive()

trading.session_token

trading.keep_alive()

trading.race_card.login()

In [43]:
source_path = '../data/daily'
destination_path = '../data/processed_daily'

# Ensure the destination folder exists
os.makedirs(destination_path, exist_ok=True)

csv_files = [f for f in os.listdir(source_path) if f.endswith('.csv')]

In [44]:
csv_files

['2024-08-22_data.csv',
 '2024-08-23_data.csv',
 '2024-08-16_data.csv',
 '2024-08-13_data.csv',
 '2024-08-26_data.csv',
 '2024-08-15_data.csv',
 '2024-08-14_data.csv']

In [45]:
def fetch_race_results(market_ids):
    try:
        data = trading.race_card.get_race_result(market_ids=market_ids)
    except Exception as e:
        print(f"An error occurred: {e}")
        data = []

    # Process and display the data
    flat_data = []

    for race in data:
        for runner in race.get('runners', []):
            for selection in runner.get('selections', []):
                if selection['marketType'] == 'WIN' and 'bsp' in selection:
                    flat_data.append({
                        'race_id': race.get('raceId'),
                        'country_code': race.get('course', {}).get('countryCode'),
                        'race_title': race.get('raceTitle'),
                        'race_class': race.get('raceClassification', {}).get('classification'),
                        'distance': race.get('distance'),
                        'course_type': race.get('course', {}).get('courseType'),
                        'surface_type': race.get('course', {}).get('surfaceType'),
                        'market_id': selection.get('marketId'),
                        'horse_id': runner.get('horseId'),
                        'saddle_cloth': runner.get('saddleCloth'),
                        'isNonRunner': runner.get('isNonRunner'),
                        'position': runner.get('position'),
                        'selection_id': selection.get('selectionId'),
                        'bsp': selection.get('bsp')
                    })

    return pd.DataFrame(flat_data)

In [48]:
merged_dfs = []

for csv in csv_files:
    df = pd.read_csv(os.path.join(source_path, csv))

    # Convert columns to string
    df['market_id'] = df['market_id'].astype(str)
    df['selection_id'] = df['selection_id'].astype(str)

    market_ids = df['market_id'].unique()

    results_df = fetch_race_results(market_ids)

    if results_df.empty:
        print(f"No data returned for market_ids: {market_ids}. Skipping this file.")
        continue  # Skip if no data is returned
    
    # Check for missing columns
    if 'market_id' not in results_df.columns or 'selection_id' not in results_df.columns:
        print(f"Expected columns missing in results. Available columns: {results_df.columns}")
        continue  # Skip if required columns are missing
    
    # Convert columns to string
    results_df['market_id'] = results_df['market_id'].astype(str)
    results_df['selection_id'] = results_df['selection_id'].astype(str)

    df_merged = df.merge(results_df, left_on=['market_id', 'selection_id'], right_on=['market_id', 'selection_id'], how='left')

    merged_dfs.append(df_merged)

    processed_file_path = os.path.join(destination_path, csv)
    df.to_csv(processed_file_path, index=False)

# Concatenate all merged DataFrames
final_df = pd.concat(merged_dfs, ignore_index=True)

final_df = final_df[final_df['status'].str.contains('ACTIVE', case=False)]

# Save the final DataFrame to a CSV file
final_df.to_csv('../data/results/results.csv', index=False)

os.remove(source_path)


In [50]:
import os
import shutil

def delete_directory_contents(directory_path):
    try:
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove the file or link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove the directory
        print(f"Successfully cleared contents of: {directory_path}")
    except Exception as e:
        print(f"Error occurred while clearing directory: {e}")

# Example usage
directory_path = '../data/daily'
delete_directory_contents(directory_path)


Successfully cleared contents of: ../data/daily
